In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Dependencies
from bs4 import BeautifulSoup
import requests
import pymongo
import dns
import datetime
from time import sleep
import random
import json

from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager

# Project defined dependencies
import config
from pprint import pprint
import cl_parser


In [3]:
# Create the craiglist parser
my_parser = cl_parser.Parser() 

In [70]:
for listing in my_parser.listings_collection.find():
    print(listing['data_id'])
    break


7278244211


In [7]:
my_parser.listings_collection.find().count()    

9069

In [9]:
find_result = my_parser.listings_collection.find_one({'data_id': '7280736862'})
pprint(find_result)

{'_id': ObjectId('60330e42566e11a3b7aedc54'),
 'created_datetime': datetime.datetime(2021, 2, 22, 1, 52, 2, 943000),
 'data_id': '7280736862',
 'listing_addrcountry': 'US',
 'listing_address': '2051 Geneva St',
 'listing_addrlocality': 'Vista',
 'listing_addrregion': 'CA',
 'listing_addrstreet': '740 Paseo Buena Vista',
 'listing_addrzip': '92085',
 'listing_attributes': ['EV charging',
                        'cats are OK - purrr',
                        'dogs are OK - wooof',
                        'apartment',
                        'w/d in unit',
                        'no smoking',
                        'detached garage'],
 'listing_availability': 'available now',
 'listing_bath': 1,
 'listing_bed': '1',
 'listing_bedbath': '1BR / 1Ba',
 'listing_datetime': '2021-02-21 11:42',
 'listing_latitude': '33.208926',
 'listing_longitude': '-117.234635',
 'listing_petsallowed': 'True',
 'listing_price': '$1,995',
 'listing_smokingallowed': 'False',
 'listing_sqft': '732ft2',
 'listi

In [6]:
my_parser.listings_collection.find().count()    

9069

In [8]:
find_result = my_parser.listings_collection.find_one({'data_id': '7280736862'})
print(find_result)

{'_id': ObjectId('60330e42566e11a3b7aedc54'), 'data_id': '7280736862', 'created_datetime': datetime.datetime(2021, 2, 22, 1, 52, 2, 943000), 'listing_title': 'Strike While the Iron is Hot! 6-Weeks FREE!', 'listing_price': '$1,995', 'listing_url': 'https://sandiego.craigslist.org/nsd/apa/d/vista-strike-while-the-iron-is-hot/7280736862.html', 'listing_address': '2051 Geneva St', 'listing_attributes': ['EV charging', 'cats are OK - purrr', 'dogs are OK - wooof', 'apartment', 'w/d in unit', 'no smoking', 'detached garage'], 'listing_availability': 'available now', 'listing_bedbath': '1BR / 1Ba', 'listing_latitude': '33.208926', 'listing_longitude': '-117.234635', 'listing_addrcountry': 'US', 'listing_addrlocality': 'Vista', 'listing_addrregion': 'CA', 'listing_addrstreet': '740 Paseo Buena Vista', 'listing_addrzip': '92085', 'listing_bath': 1, 'listing_bed': '1', 'listing_petsallowed': 'True', 'listing_smokingallowed': 'False', 'listing_type': 'Apartment', 'listing_sqft': '732ft2', 'listin

In [10]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324
[WDM] - Driver [C:\Users\melis\.wdm\drivers\chromedriver\win32\88.0.4324.96\chromedriver.exe] found in cache




In [ ]:
# iterate through the list of URLs, return each region listing and then run through the pages
if not config.test:
    for url in urls:
        browser.visit(url)

        # Iterate through all pages
        for x in range(config.url_page_range):
            # HTML object
            html_listings = browser.html
            # Parse HTML with Beautiful Soup
            soup_listings = BeautifulSoup(html_listings, 'html.parser')
            # Retrieve all elements that contain book information
            soup_listings = BeautifulSoup(html_listings)

            results = soup_listings.find_all('li', class_='result-row')

            # Loop through returned results
            for result in results:
                try:
                    my_parser.insert_listing(result)                                                                  
                except Exception as e:
                    print(e)        
            try:
                browser.links.find_by_partial_text('next').click()          
            except:
                print("Scraping Complete")
                break

In [11]:
x = 0

for l in my_parser.listings_collection.find({'listing_addrzip':{"$exists":False}}):        
  
    data_id = l['data_id']
    url_details = l['listing_url']
    
    print('Index ' + str(x) + ' Updating listing details data_id ' + data_id)

    if (url_details != ''):
        sleep(random.randint(1,3))
        browser.visit(url_details)
        html_details = browser.html
        
        soup_details = BeautifulSoup(html_details)
        # if soup_details.find('div',class_='removed') == 'None':            
        try:
            my_parser.insert_listing_details(soup_details, data_id)
        except Exception as e:               
            print(e)                  
    x += 1
    #if x > 200:            
    #    print('Scrape Completed')
    #    break

          
        


21 Park Way
Apartment
1
1

False
------------------------------
Index 904 Updating listing details data_id 7281695642
7281695642
32.822499
-117.178301
2BR / 2Ba
925ft2
available now
['cats are OK - purrr', 'dogs are OK - wooof', 'apartment', 'laundry on site', 'no smoking', 'carport']
US
San Diego
CA
92117
5550 Balboa Arms Dr
Apartment
2
2
True
False
------------------------------
Index 905 Updating listing details data_id 7281690271
7281690271
33.132921
-117.094667
1BR / 1Ba
668ft2
available now
['cats are OK - purrr', 'dogs are OK - wooof', 'apartment', 'laundry in bldg', 'carport']
US
Escondido
CA
92033
1045 Morning View Drive
Apartment
1
1
True

------------------------------
Index 906 Updating listing details data_id 7281692707
7281692707
32.617357
-116.987572
1BR / 1Ba
732ft2
available now
['EV charging', 'application fee details: $25.00 Per Applicant', 'cats are OK - purrr', 'dogs are OK - wooof', 'apartment', 'w/d in unit', 'no smoking', 'detached garage']
US
Chula Vista
CA
919